In [16]:
import pandas as pd
import json
from collections import Counter, defaultdict
import zipfile
import os

# loading csv (from letterboxd, tracks watched movies)
zip_path = r"C:\Users\maria.DESKTOP-4LV638A\Downloads\letterboxd-bibblybop-2026-02-21-02-24-utc.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv') and 'watched' in f.lower()]
    if not csv_files:
        raise FileNotFoundError("no watched.csv found in the zip!")
    
    csv_name = csv_files[0]
    zip_ref.extract(csv_name, os.getcwd())

# loading into dataframe
df_watched = pd.read_csv(csv_name)

# test
# print(df_watched.head())
watched_movies = df_watched[['Date', 'Name', 'Year', 'Letterboxd URI']].to_dict(orient="records")
# print(watched_movies[:5])

# filtering data by date (since taste for movies typically concentrates in different months)
df_watched['Date'] = pd.to_datetime(df_watched['Date'], errors='coerce')
df_watched['Year'] = pd.to_numeric(df_watched['Year'], errors='coerce')
df_watched['Month'] = df_watched['Date'].dt.month

# test
# target_month = 10
# df_month = df_watched[df_watched['Month'] == target_month]
# print(f"Movies watched in month {target_month}: {len(df_month)}")
# print(df_month[['Name', 'Date','Year']].head())

# df_trainingdata = df_watched[df_watched['Date'].dt.year >= 2025]
# month_counts = df_trainingdata['Month'].value_counts().sort_index()
# print(month_counts)

df_trainingdata = df_watched[df_watched['Date'].dt.year >= 2025].copy()
# calculate continuous month index
df_trainingdata['MonthIndex'] = (df_trainingdata['Date'].dt.year - 2025) * 12 + df_trainingdata['Date'].dt.month

# test
print(df_trainingdata.head(15))

watched_keys = set((row['Name'], row['Year']) for row in df_watched.to_dict(orient="records"))


          Date                                   Name  Year  \
538 2025-01-01  The Ministry of Ungentlemanly Warfare  2024   
539 2025-01-01                      L.A. Confidential  1997   
540 2025-01-02                               Juror #2  2024   
541 2025-01-04                               Scrooged  1988   
542 2025-01-07               Tucker and Dale vs. Evil  2010   
543 2025-01-08                   Crazy, Stupid, Love.  2011   
544 2025-01-09           How to Lose a Guy in 10 Days  2003   
545 2025-01-12                           The Fall Guy  2024   
546 2025-01-13                           Spider-Man 3  2007   
547 2025-01-16             O Brother, Where Art Thou?  2000   
548 2025-01-27                            The Patriot  2000   
549 2025-02-08                              Set It Up  2018   
550 2025-03-02                                   Flow  2024   
551 2025-03-02                         The Apprentice  2024   
552 2025-03-17                                     EO  

In [4]:
# reading in imdb data on ratings, keeping only the movies identification, avg rating, and num of votes
df_ratings = pd.read_csv(
    "title.ratings.tsv",
    sep='\t',
    usecols=['tconst', 'averageRating', 'numVotes'],
    dtype={'tconst':str}
)

df_ratings['averageRating'] = pd.to_numeric(df_ratings['averageRating'], errors='coerce')
df_ratings['numVotes'] = pd.to_numeric(df_ratings['numVotes'], errors='coerce')


# keeping only movies where the average rating is at least 7 with at least 2000 votes
df_ratings = df_ratings[(df_ratings['averageRating']>=7) & (df_ratings['numVotes'] >= 1000)]

print(f"There are {len(df_ratings)} movies in our rating df.")
df_ratings.head()

There are 58160 movies in our rating df.


,tconst,averageRating,numVotes
11,tt0000012,7.4,13912
13,tt0000014,7.1,6367
208,tt0000211,7.4,5285
333,tt0000359,7.1,3497
369,tt0000417,8.1,60334


In [5]:
# reading in imdb data on movie info in chunks, keeping identification, type of media, title of media, release year, genres
chunks = []

for chunk in pd.read_csv(
    'title.basics.tsv',
    sep='\t',
    usecols = ['tconst', 'titleType', 'primaryTitle', 'startYear', 'genres'],
    dtype='str',
    chunksize=100_000
):
    # only keep movies and tvseries
    chunk = chunk[chunk['titleType'].isin(['movie','tvseries'])]
    # only keep movies that satisfy rating requirement
    # chunk = chunk[chunk['tconst'].isin(df_ratings['tconst'])]

    chunk = chunk[["tconst","titleType","primaryTitle","startYear","genres"]]

    chunks.append(chunk)


# combine chunks into df
df_movies = pd.concat(chunks, ignore_index=True)

# use same naming convention as watched.csv
df_movies = df_movies.rename(columns={"primaryTitle":"Name", "startYear":"Year"})

# split genres into lists
df_movies['genres'] = df_movies['genres'].apply(lambda x: x.split(",") if pd.notna(x) else [])
# convert Year to numeric (precaution)
df_movies['Year'] = pd.to_numeric(df_movies['Year'], errors='coerce')

print(len(df_movies))
print(df_movies.head())


738507
      tconst titleType                           Name    Year  \
0  tt0000009     movie                     Miss Jerry  1894.0   
1  tt0000147     movie  The Corbett-Fitzsimmons Fight  1897.0   
2  tt0000502     movie                       Bohemios  1905.0   
3  tt0000574     movie    The Story of the Kelly Gang  1906.0   
4  tt0000591     movie               The Prodigal Son  1907.0   

                           genres  
0                       [Romance]  
1      [Documentary, News, Sport]  
2                            [\N]  
3  [Action, Adventure, Biography]  
4                         [Drama]  


In [ ]:
# add data of ratings to watched csv

# drop rows with missing Name or Year
df_watched = df_watched.dropna(subset=['Name','Year'])
df_movies = df_movies.dropna(subset=['Name','Year'])

# key for watched movies and imbd titles
df_watched['Key'] = list(zip(df_watched['Name'], df_watched['Year']))
df_movies['Key'] = list(zip(df_movies['Name'], df_movies['Year']))
df_movies_filtered['Key'] = list(zip(df_movies_filtered['Name'], df_movies_filtered['Year']))

# adding genres to the watched df
df_watched_with_genres = df_watched.merge(
    df_movies[['Key', 'genres']],  # only keep the key and genres
    on='Key',
    how='left'  # keep all watched movies even if no genre found
)

print(df_watched_with_genres.head(10))

        Date                                               Name  Year  \
0 2024-06-05                                        Challengers  2024   
1 2024-06-05  The Hunger Games: The Ballad of Songbirds & Sn...  2023   
2 2024-06-05                                               Soul  2020   
3 2024-06-05                                              Tenet  2020   
4 2024-06-05                              Promising Young Woman  2020   
5 2024-06-05                                             Minari  2020   
6 2024-06-05                                          Nomadland  2020   
7 2024-06-05                                             Onward  2020   
8 2024-06-05                                              Emma.  2020   
9 2024-06-05                                         The Father  2020   

         Letterboxd URI  Month  \
0  https://boxd.it/zld0      6   
1  https://boxd.it/pZCY      6   
2  https://boxd.it/ioLA      6   
3  https://boxd.it/leq4      6   
4  https://boxd.it/loRE   

In [12]:
df_ratings_renamed = df_ratings.rename(columns={
    'averageRating': 'rating',
    'numVotes': 'votes'
})

print(df_ratings.columns.tolist())
print(df_movies.head())
df_movies_filtered = df_movies.merge(
    df_ratings_renamed[['tconst','rating','votes']],
    on='tconst',
    how='inner'
)

print(f"{len(df_movies_filtered)}")

df_movies_filtered.head()

['tconst', 'averageRating', 'numVotes']
      tconst titleType                           Name    Year  \
0  tt0000009     movie                     Miss Jerry  1894.0   
1  tt0000147     movie  The Corbett-Fitzsimmons Fight  1897.0   
2  tt0000502     movie                       Bohemios  1905.0   
3  tt0000574     movie    The Story of the Kelly Gang  1906.0   
4  tt0000591     movie               The Prodigal Son  1907.0   

                           genres                                      Key  
0                       [Romance]                     (Miss Jerry, 1894.0)  
1      [Documentary, News, Sport]  (The Corbett-Fitzsimmons Fight, 1897.0)  
2                            [\N]                       (Bohemios, 1905.0)  
3  [Action, Adventure, Biography]    (The Story of the Kelly Gang, 1906.0)  
4                         [Drama]               (The Prodigal Son, 1907.0)  
13898


,tconst,titleType,Name,Year,genres,Key,rating,votes
0,tt0002130,movie,Dante's Inferno,1911.0,"[Adventure, Drama, Fantasy]","(Dante's Inferno, 1911.0)",7.1,4048
1,tt0003014,movie,Ingeborg Holm,1913.0,[Drama],"(Ingeborg Holm, 1913.0)",7.0,1589
2,tt0003740,movie,Cabiria,1914.0,"[Adventure, Drama, History]","(Cabiria, 1914.0)",7.1,4306
3,tt0006206,movie,Les Vampires,1915.0,"[Action, Adventure, Crime]","(Les Vampires, 1915.0)",7.3,5857
4,tt0006864,movie,Intolerance,1916.0,"[Drama, History]","(Intolerance, 1916.0)",7.6,17849


In [74]:
# get set of watched movie keys
watched_keys = set(df_watched_with_genres['Key'])

# filter df_movies_filtered to exclude already watched movies
df_movies_filtered = df_movies_filtered[~df_movies_filtered['Key'].isin(watched_keys)].copy()

print(f"Number of candidate movies after excluding watched: {len(df_movies_filtered)}")
print(df_movies_filtered.head())


# check/debugging
if ("The Shining", 1980) in set(df_movies_filtered['Key']):
    print("Uh oh: The Shining is STILL in the candidate movies!")
else:
    print("Yippee: The Shining has been successfully excluded.")

Number of candidate movies after excluding watched: 13500
      tconst titleType             Name    Year                       genres  \
0  tt0002130     movie  Dante's Inferno  1911.0  [Adventure, Drama, Fantasy]   
1  tt0003014     movie    Ingeborg Holm  1913.0                      [Drama]   
2  tt0003740     movie          Cabiria  1914.0  [Adventure, Drama, History]   
3  tt0006206     movie     Les Vampires  1915.0   [Action, Adventure, Crime]   
4  tt0006864     movie      Intolerance  1916.0             [Drama, History]   

                         Key  rating  votes  
0  (Dante's Inferno, 1911.0)     7.1   4048  
1    (Ingeborg Holm, 1913.0)     7.0   1589  
2          (Cabiria, 1914.0)     7.1   4306  
3     (Les Vampires, 1915.0)     7.3   5857  
4      (Intolerance, 1916.0)     7.6  17849  
Yippee: The Shining has been successfully excluded.


In [64]:
print("Training data df:")
print(df_trainingdata.head())

print("\n\nMovies filtered df:")
print(df_movies_filtered.head())

print("\n\nWatched with genres df:")
print(df_watched_with_genres.head())


Training data df:
          Date                                   Name  Year  \
538 2025-01-01  The Ministry of Ungentlemanly Warfare  2024   
539 2025-01-01                      L.A. Confidential  1997   
540 2025-01-02                               Juror #2  2024   
541 2025-01-04                               Scrooged  1988   
542 2025-01-07               Tucker and Dale vs. Evil  2010   

           Letterboxd URI  Month  MonthIndex  
538  https://boxd.it/u5zu      1           1  
539  https://boxd.it/27oC      1           1  
540  https://boxd.it/FRwU      1           1  
541  https://boxd.it/1Y9e      1           1  
542   https://boxd.it/Mce      1           1  


Movies filtered df:
      tconst titleType             Name    Year                       genres  \
0  tt0002130     movie  Dante's Inferno  1911.0  [Adventure, Drama, Fantasy]   
1  tt0003014     movie    Ingeborg Holm  1913.0                      [Drama]   
2  tt0003740     movie          Cabiria  1914.0  [Adventure

In [ ]:
# Create a new dataframe for building monthly genre profiles
df_monthly_profiles_input = df_watched_with_genres[df_watched_with_genres['Date'].dt.year >= 2025].copy()

# Compute a continuous month index (Jan 2025 = 1, Feb 2025 = 2, ..., Jan 2026 = 13, etc.)
df_monthly_profiles_input['MonthIndex'] = (
    (df_monthly_profiles_input['Date'].dt.year - 2025) * 12
    + df_monthly_profiles_input['Date'].dt.month
)

# Keep only the relevant columns for monthly genre profiling
df_monthly_profiles_input = df_monthly_profiles_input[['MonthIndex', 'genres', 'Letterboxd URI']].copy()

# rename letterboxd uri id to id
df_monthly_profiles_input.rename(columns={'Letterboxd URI': 'id'}, inplace=True)

# Quick check
print(df_monthly_profiles_input.head(20))

     MonthIndex                          genres                    id
554           1           [Action, Comedy, War]  https://boxd.it/u5zu
555           1         [Crime, Drama, Mystery]  https://boxd.it/27oC
556           1         [Crime, Drama, Mystery]  https://boxd.it/FRwU
557           1        [Comedy, Drama, Fantasy]  https://boxd.it/1Y9e
558           1                             NaN   https://boxd.it/Mce
559           1        [Comedy, Drama, Romance]   https://boxd.it/D9A
560           1               [Comedy, Romance]  https://boxd.it/1XwG
561           1         [Action, Comedy, Drama]  https://boxd.it/s0Bm
562           1     [Action, Adventure, Sci-Fi]  https://boxd.it/2a7Y
563           1      [Adventure, Comedy, Crime]  https://boxd.it/2b3I
564           1            [Action, Drama, War]  https://boxd.it/27Bw
565           2               [Comedy, Romance]  https://boxd.it/dpik
566           3  [Adventure, Animation, Family]  https://boxd.it/v1Ei
567           3     

In [66]:
def build_monthly_genre_profiles(df):
    """
    df: DataFrame with columns 'MonthIndex' and 'genres'
    returns: dict mapping MonthIndex -> Counter of genres
    """
    month_profiles = defaultdict(Counter)

    for _, row in df.iterrows():
        month = row['MonthIndex']
        genres = row['genres']

        # ensure genres is a list
        if isinstance(genres, list):
            genres_list = genres
        elif isinstance(genres, str):
            # convert to list just in case
            genres_list = [g.strip() for g in genres.strip("[]").split(",") if g.strip()]
        else:
            # otherwise empty list
            genres_list = []

        for g in genres_list:
            month_profiles[month][g] += 1  # count each genre

    return month_profiles

# creating profiles
month_profiles = build_monthly_genre_profiles(df_monthly_profiles_input)

# check
# import pprint
# months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# for i in range(1,13):
#     print(f"{months[i-1]} 2025 Genre Counter:")
#     pprint.pprint(dict(month_profiles[i]))


# print(month_profiles[9])
print(month_profiles[13])

Counter({'Comedy': 8, 'Drama': 6, 'Action': 6, 'Crime': 5, 'Adventure': 4, 'Romance': 3, 'Sci-Fi': 3, 'Horror': 3, 'History': 1, 'War': 1, 'Thriller': 1, 'Family': 1, 'Animation': 1, 'Fantasy': 1})


In [83]:
import random
import numpy as np

def recommend_movies_for_month(month_index, month_profiles, df_movies, top_n=200, select_n=8):
    """
    month_index: int, MonthIndex for which we want recommendations
    month_profiles: dict, MonthIndex -> Counter of genres
    df_movies: DataFrame, candidate movies with 'genres' column
    top_n: int, how many top similar movies to consider
    select_n: int, how many movies to randomly pick for final recommendation
    """
    
    # Get month vector
    month_counter = month_profiles.get(month_index)
    if not month_counter:
        print(f"No data for month {month_index}")
        return []
    
    # All genres seen across this month
    genre_space = list(month_counter.keys())
    
    # convert month counter to vector
    month_vector = np.array([month_counter[g] for g in genre_space], dtype=float)
    month_vector /= month_vector.sum()  # normalize to probability vector
    
    # Compute similarity for each movie
    movie_scores = []
    
    for idx, row in df_movies.iterrows():
        movie_genres = row['genres']
        # create movie vector aligned to genre_space
        movie_vector = np.array([1 if g in movie_genres else 0 for g in genre_space], dtype=float)
        if movie_vector.sum() == 0:
            continue  # skip movies with no overlap
        movie_vector /= movie_vector.sum()  # normalize
        
        # similarity score = dot product
        movie_scores.append((np.dot(month_vector, movie_vector), idx))
    
    # sort by similarity descending
    movie_scores.sort(reverse=True, key=lambda x: x[0])
    
    # pick top N movies
    top_movies_idx = [idx for _, idx in movie_scores[:top_n]]
    
    # randomly select select_n movies from top_movies
    recommended_idx = random.sample(top_movies_idx, min(select_n, len(top_movies_idx)))
    
    return df_movies.loc[recommended_idx]

# Example: recommend for October 2025 (MonthIndex = 10)
recommended_oct = recommend_movies_for_month(14, month_profiles, df_movies_filtered)
print(recommended_oct[['Name', 'genres', 'rating']])

                   Name                       genres  rating
791             Panique               [Crime, Drama]     7.9
390         Kid Galahad    [Crime, Drama, Film-Noir]     7.2
429        Boys' School             [Drama, Mystery]     7.3
161       The Big House     [Crime, Drama, Thriller]     7.1
608           Kings Row                      [Drama]     7.5
689               Laura  [Drama, Film-Noir, Mystery]     7.9
127  The Man Who Laughs     [Drama, Horror, Mystery]     7.6
18          The Penalty       [Crime, Drama, Horror]     7.3
